In [1]:
import os
import copy
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_california_housing, make_regression
from sklearn.model_selection import train_test_split
housing = fetch_california_housing()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  

X_train, X_test, y_train, y_test = train_test_split(housing.data, housing.target, test_size=0.98, random_state=2)
XR, yR = make_regression(n_samples=100, n_features=2, noise=0, random_state=5)
X_train2, X_test2, y_train2, y_test2 = train_test_split(XR, yR, test_size=0.20, random_state=2)

def normalize_dataset(X):
    return tf.keras.utils.normalize(X)

c:\Users\Utente\AppData\Local\Programs\Python\Python39\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
class LinearRegressor(tf.Module):
    def __init__(self, layers, X, y, epochs=100, lr=0.001):
        self.layers = layers
        self.X = X
        self.lr = lr
        self.y = y
        self.optimizer = tf.keras.optimizers.Adam(lr)
        self.mse = tf.keras.metrics.MeanSquaredError()
        self.accuracy = tf.keras.metrics.MeanSquaredLogarithmicError()
        self.precision = tf.keras.metrics.Precision()
    
    @tf.function
    def train_step(self):
        return tf.multiply(tf.multiply(tf.linalg.inv(tf.multiply(tf.transpose(self.X), self.X)), tf.transpose(self.X)), self.y)

model = LinearRegressor(X_train2, y_train2, 1, 0.001)
print(model.train_step())


ValueError: in user code:

    File "C:\Users\Utente\AppData\Local\Temp\ipykernel_19296\7959388.py", line 14, in train_step  *
        return tf.multiply(tf.multiply(tf.linalg.inv(tf.matmul(tf.transpose(self.X), self.X)), tf.transpose(self.X)), self.y)

    ValueError: Shape must be rank 2 but is rank 1 for '{{node MatMul}} = MatMul[T=DT_DOUBLE, transpose_a=false, transpose_b=false](transpose, MatMul/b)' with input shapes: [80], [80].
